# Kaggle Agents + MLE-bench Lite (Colab)

Este notebook executa o `kaggle-agents` em competições do **MLE-bench Lite** e valida as submissões com `mlebench grade-sample`.

Pré-requisitos:
- `OPENAI_API_KEY`
- Credenciais Kaggle (`KAGGLE_USERNAME`, `KAGGLE_KEY`) para `mlebench prepare` (download dos datasets)


## 1) Instalação

Opcional: clone os repositórios (requer rede). Se você já subiu os arquivos no Colab, pule o `git clone`.

In [ ]:
# Kaggle Agents
# !git clone https://github.com/<seu-user>/kaggle-agents.git /content/kaggle-agents
%cd /content/kaggle-agents
!pip -q install -e .

# MLE-bench (requer rede)
# !git clone https://github.com/openai/mle-bench.git /content/mle-bench
# !pip -q install -e /content/mle-bench

import sys
print('python:', sys.version)

## 2) Configurar credenciais (Colab Secrets)

No Colab, vá em **Secrets** e crie as chaves:
- `OPENAI_API_KEY`
- `KAGGLE_USERNAME`
- `KAGGLE_KEY`

In [ ]:
import os, json
from pathlib import Path

try:
    from google.colab import userdata
except Exception as e:
    raise RuntimeError('Este notebook foi feito para rodar no Google Colab.') from e

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY') or ''

# Kaggle credentials para o CLI do mlebench
kaggle_username = userdata.get('KAGGLE_USERNAME') or ''
kaggle_key = userdata.get('KAGGLE_KEY') or ''

if not os.environ['OPENAI_API_KEY']:
    raise ValueError('Missing OPENAI_API_KEY (configure nos Colab Secrets).')
if not kaggle_username or not kaggle_key:
    raise ValueError('Missing KAGGLE_USERNAME/KAGGLE_KEY (configure nos Colab Secrets).')

kaggle_path = Path.home() / '.kaggle' / 'kaggle.json'
kaggle_path.parent.mkdir(parents=True, exist_ok=True)
kaggle_path.write_text(json.dumps({'username': kaggle_username, 'key': kaggle_key}))
kaggle_path.chmod(0o600)

print('✅ Secrets configurados e kaggle.json escrito em:', kaggle_path)

## 3) Preparar datasets do MLE-bench Lite

Isso baixa e prepara as 22 competições do Lite. Pode levar algum tempo.

Se você quiser testar rápido, rode apenas uma competição:
- `mlebench prepare -c aerial-cactus-identification`

In [ ]:
# Lite completo
# !mlebench prepare --lite

# Teste rápido (1 competição)
!mlebench prepare -c aerial-cactus-identification

## 4) Rodar avaliação (gera `results.json`, `summary.json`, `results.csv`)

Dica para economizar tempo/custo:
- Comece com `--max-iterations 1` e `--timeout 1200` (20 min por componente)
- Depois aumente gradualmente.


In [ ]:
!python notebooks/mlebench_eval.py --competition aerial-cactus-identification --max-iterations 1 --timeout 1800 -o /content/mlebench_results

In [ ]:
import json
from pathlib import Path

out_dir = Path('/content/mlebench_results')
summary = json.loads((out_dir / 'summary.json').read_text())
print('Summary:', json.dumps(summary, indent=2))
